<a href="https://colab.research.google.com/github/sophiasagan/DSI_Repo/blob/main/Basic_RNN_Stock_Predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import yfinance as yf
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense
from sklearn.model_selection import train_test_split

In [2]:
def fetch_and_preprocess(symbol, start_date, end_date):
    # Fetch stock data and preprocess
    data = yf.download(symbol, start=start_date, end=end_date)
    data['Close'] = data['Close'].values
    return data

In [3]:
def create_dataset(df, look_back=1):
    # Creating the dataset with the look-back time period.
    X, Y = [], []
    for i in range(len(df)-look_back-1):
        a = df[i:(i+look_back)]
        X.append(a)
        Y.append(df[i + look_back])
    return np.array(X).reshape(-1, look_back, 1), np.array(Y)

In [4]:
def create_rnn_model(input_shape):
    # Setting up the model and adding a few layers.
    model = Sequential()
    model.add(SimpleRNN(64, return_sequences=True, activation='tanh', input_shape=input_shape))
    model.add(SimpleRNN(64, return_sequences=False, activation='tanh'))
    model.add(Dense(32))
    model.add(Dense(1))
    return model

In [6]:
def predict_next_day_close_price(model, last_30_days):
    # Reshaping the data to be used for the model prediction and getting the actual model prediction.
    X_pred = np.reshape(last_30_days, (1, 30, 1))
    predicted_price = model.predict(X_pred)
    return predicted_price[0][0]

In [18]:
def main(ticker):
    # Fetch and preprocess data
    data = fetch_and_preprocess(ticker, '2020-01-01', '2024-01-08')

    # Prepare and split data
    X, Y = create_dataset(data['Close'].values, look_back=30)

    # Reshape input to be [samples, time steps, features]
    X_train, _, Y_train, _ = train_test_split(X, Y, test_size=0.2, random_state=42)

    # Create the RNN model and compile it
    model = create_rnn_model((30, 1))
    model.compile(optimizer='adam', loss='mean_squared_error')

    # Fit the model
    model.fit(X_train, Y_train, epochs=100, batch_size=64)

    # Predict the next day close price
    last_30_days = data['Close'][-30:].values
    predicted_price = predict_next_day_close_price(model, last_30_days)
    predicted_price = round(predicted_price, 2)

    # Printing the output.
    print(f"The predicted next day closing price for {ticker} is: {predicted_price:.2f}")

In [19]:
if __name__ == "__main__":
    main('GOOGL')

[*********************100%%**********************]  1 of 1 completed
Epoch 1/100
13/13 [==============================] - 2s 16ms/step - loss: 11577.8643
Epoch 2/100
13/13 [==============================] - 0s 16ms/step - loss: 10073.3525
Epoch 3/100
13/13 [==============================] - 0s 17ms/step - loss: 8771.3340
Epoch 4/100
13/13 [==============================] - 0s 16ms/step - loss: 7445.0132
Epoch 5/100
13/13 [==============================] - 0s 16ms/step - loss: 6093.1079
Epoch 6/100
13/13 [==============================] - 0s 16ms/step - loss: 4784.6743
Epoch 7/100
13/13 [==============================] - 0s 17ms/step - loss: 3587.3276
Epoch 8/100
13/13 [==============================] - 0s 16ms/step - loss: 2586.9038
Epoch 9/100
13/13 [==============================] - 0s 18ms/step - loss: 1813.8391
Epoch 10/100
13/13 [==============================] - 0s 29ms/step - loss: 1256.7769
Epoch 11/100
13/13 [==============================] - 0s 28ms/step - loss: 928.7130
Epoc